In [44]:
import numpy as np
from scipy.linalg import expm
from qiskit import *
import pandas as pd

In [52]:
url = "https://raw.githubusercontent.com/ibonreinoso/qiskit-hackathon-bilbao-19/master/DAX_PERFORMANCE_INDEX.csv"
data = pd.read_csv(url, sep=';')

In [53]:
data = data.drop(['wkn_500340'], axis = 1)
data = data.loc[:,['wkn_515100', 'wkn_575200']]
print(data)

      wkn_515100  wkn_575200
0         21.450       22.67
1         21.780       23.35
2         21.750       23.37
3         22.240       23.69
4         22.290       23.70
5         22.025       23.70
6         21.900       23.60
7         21.935       23.45
8         21.975       23.36
9         21.675       23.05
10        21.925       22.93
11        22.025       22.98
12        21.950       22.85
13        21.900       22.61
14        21.950       22.62
15        22.000       22.52
16        21.950       22.35
17        21.935       22.21
18        21.775       21.85
19        21.825       22.20
20        21.950       22.20
21        21.575       21.80
22        21.550       21.58
23        21.275       21.30
24        21.600       21.48
25        21.625       21.21
26        22.095       21.94
27        22.100       22.08
28        22.035       21.98
29        22.225       22.00
...          ...         ...
1077      39.390       53.94
1078      39.470       54.51
1079      39.9

In [66]:
sigma2 = np.cov(data.values.T)
rho2 = sigma2 /np.matrix.trace(sigma2)
print(rho2)

[[0.30676853 0.44383242]
 [0.44383242 0.69323147]]


In [72]:
eigenvalues,(eigenvector1, eigenvector2)= np.linalg.eigvalsh(rho2)
print(eigenvalues, eigenvector1, eigenvector2)


TypeError: cannot unpack non-iterable numpy.float64 object

In [68]:
eigenvector1.dot(rho2)

array([0.50049145, 0.75225124])

In [69]:
eigenvector1 * eigenvalues[0]

array([0.01329469, 0.00877241])

In [70]:
NUM_QUBITS = 3
NUM_ITERATION = 1000
ACCURACY = 0.1
SHOTS_PER_ITERATION = 8192
backend = BasicAer.get_backend('qasm_simulator')

state_vector = [ 1/np.sqrt(2), 1/np.sqrt(2)]
iterate = True
for i in range(0, 100):
    quantum_circuit = QuantumCircuit(NUM_QUBITS, NUM_QUBITS)
    quantum_circuit.initialize(state_vector, NUM_QUBITS-1)
    quantum_circuit.h(0)
    quantum_circuit.h(1)
    (th1, ph1, lam1) = qiskit.quantum_info.synthesis.two_qubit_decompose.euler_angles_1q(expm(2*1j*np.pi*rho2))
    quantum_circuit.cu3(th1, ph1, lam1, 1, 2)
    (th2, ph2, lam2) = qiskit.quantum_info.synthesis.two_qubit_decompose.euler_angles_1q(expm(2*1j*np.pi*rho2*2))
    quantum_circuit.cu3(th2, ph2, lam2, 0, 2)
    quantum_circuit.h(0)
    quantum_circuit.crz(-np.pi/2,0,1)
    quantum_circuit.h(1)
    quantum_circuit.measure([0,1,2],[0,1,2])
    results = execute(quantum_circuit, backend=backend, shots=SHOTS_PER_ITERATION).result().get_counts()
    denominator_result = results['111'] + results['011']
    alpha1 = np.sqrt(results['011'] / denominator_result)
    alpha2 = np.sqrt(results['111'] / denominator_result)
    new_state = [alpha1, alpha2]
    # check the acc 
    state_vector = new_state

In [71]:
print(state_vector, eigenvector1)

[0.5645186610567614, 0.825420305855557] [0.83466977 0.55075073]
